In [5]:
#from astropy.io import fits
import numpy as np
#import glob
#import os
import pickle
import matplotlib.pyplot as plt
import astropy
import pandas as pd
#from astropy.io import fits
#from scipy.optimize import curve_fit

from photutils.centroids import (centroid_1dg, centroid_2dg,
                                 centroid_com, centroid_quadratic)

%matplotlib qt

In [2]:
# retrieve processed data arrays from pickle files

# single fiber
file = open('yoo_jung_single.pkl', 'rb')
dict_single = pickle.load(file)
file.close()

# 3 photonic lantern
file = open('yoo_jung_3PL.pkl', 'rb')
dict_3PL = pickle.load(file)
file.close()

# broadband image
file = open('yoo_jung_broadband.pkl', 'rb')
dict_broadband = pickle.load(file)
file.close()

In [3]:
# retrieve centroids as fcn of wavelength

file = open('single_wavel_centroids.pkl', 'rb')
dict_centroids = pickle.load(file)
file.close()

In [20]:
# convert to DataFrame 

df_centroids = pd.DataFrame(columns=['wavel','x','y'])

for key, val in dict_centroids.items():
    
    dict_this = {'wavel':float(key),'x':val[0],'y':val[1]}
    
    df_centroids = df_centroids.append(dict_this, ignore_index=True)

85.03786377864677
{'wavel': 1100.0, 'x': 85.03786377864677, 'y': 131.89772826556515}
301.9801788491584
{'wavel': 1680.0, 'x': 301.9801788491584, 'y': 133.23213765670403}
215.30878252514378
{'wavel': 1340.0, 'x': 215.30878252514378, 'y': 132.89481839956434}
222.86945844527565
{'wavel': 1360.0, 'x': 222.86945844527565, 'y': 132.9232897890987}
100.20426046850436
{'wavel': 1120.0, 'x': 100.20426046850436, 'y': 131.9618274003124}
127.71033132256613
{'wavel': 1160.0, 'x': 127.71033132256613, 'y': 132.0609486150821}
207.91323484171176
{'wavel': 1320.0, 'x': 207.91323484171176, 'y': 132.8722124831927}
259.2472425510047
{'wavel': 1480.0, 'x': 259.2472425510047, 'y': 133.04631243887036}
199.8631324058905
{'wavel': 1300.0, 'x': 199.8631324058905, 'y': 132.84157355680244}
114.51902504280329
{'wavel': 1140.0, 'x': 114.51902504280329, 'y': 132.01547232078838}
310.0845150108871
{'wavel': 1720.0, 'x': 310.0845150108871, 'y': 133.75680910662643}
278.7704063561162
{'wavel': 1560.0, 'x': 278.770406356116

/var/folders/8y/ztvmc5ys1pn64bjywt1qp4k40000gs/T/ipykernel_2883/810300173.py:11: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_centroids = df_centroids.append(dict_this, ignore_index=True)
/var/folders/8y/ztvmc5ys1pn64bjywt1qp4k40000gs/T/ipykernel_2883/810300173.py:11: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_centroids = df_centroids.append(dict_this, ignore_index=True)
/var/folders/8y/ztvmc5ys1pn64bjywt1qp4k40000gs/T/ipykernel_2883/810300173.py:11: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_centroids = df_centroids.append(dict_this, ignore_index=True)
/var/folders/8y/ztvmc5ys1pn64bjywt1qp4k40000gs/T/ipykernel_2883/810300173.py:11: FutureWarning: The frame.append method is deprecated and will be removed fro

In [21]:
df_centroids

,wavel,x,y
0,1100.0,85.037864,131.897728
1,1680.0,301.980179,133.232138
2,1340.0,215.308783,132.894818
3,1360.0,222.869458,132.923290
4,1120.0,100.204260,131.961827
5,1160.0,127.710331,132.060949
6,1320.0,207.913235,132.872212
7,1480.0,259.247243,133.046312
8,1300.0,199.863132,132.841574
9,1140.0,114.519025,132.015472


In [5]:
df_centroids

NameError: name 'df_centroids' is not defined